### WT.cluster.avg_module.avg 220112

In [ ]:
wd<-"/home/jovyan/zxli_SCI/result/WGCNA/WT.SCT.res02.domain_220311/"#"/home/jovyan/zxli_SCI/result/WGCNA/spot/s4_allgene/"
#wd<-"/home/jovyan/zxli_SCI/result/WGCNA/WTKO/spot/"
#dir.create(wd)
setwd(wd)
getwd()

In [99]:
library(Seurat)
library(ggplot2)
library(patchwork)
library(dplyr)
library(hdf5r)
library(png)
library(ggalluvial)
#library(tidyverse)
library(clustree)
library(RCTD)
library(Matrix)
library(SPOTlight)
library(future)
library(grid)
library(RColorBrewer)
library(pheatmap)
library(circlize)
library(igraph)
library(ComplexHeatmap)
library(stringr)
library(ComplexHeatmap)
library(cowplot)
library(biomaRt)
#library(scClustViz)
library(ggsci)
library(colorspace)
library(scales)

In [ ]:
da<-readRDS("/home/jovyan/zxli_SCI/result/Seurat/reg.CC/WT_replace_v2/res02_220310/WT.merge.replace_v2.SCT.regress_CC.nC.mt.ident.pc20.k50.res02.rds")
da

In [112]:
write.csv(rownames(da),"/home/jovyan/zxli_SCI/data/Seurat/WT.SCT.allgenes.csv")

### module gene average expression on space

In [ ]:
### correlation module 220117
#modgene<-read.csv("/home/jovyan/zxli_SCI/result/WGCNA/spot/pearson/WTKO.s4.SCT.data.genotype.top1w.hvg.union_pearson.correlation.L1norm.averagelinkage.k200.filt30.27module_gene.csv")
modgene<-read.csv("thr100.sp18.hclust_average.min30.deep2.h015.34modules.full.gene.list.csv")#("min30.ch09.mh01/min30.cuth09.mergh01.modules.full.gene.list.csv")#("WT.s4.modules.mergheight045.full.gene.list.csv")#read.csv("WT.s4.hvg1w.modules.full.gene.list.csv")
rownames(modgene)<-modgene$gene
dim(modgene)
modgene<-modgene[!modgene$module=="grey",]
#modgene<-modgene[,-which(colnames(modgene)%in%c("X","gene"))]
head(modgene)
dim(modgene)
table(modgene$module)
length(unique(modgene$module))

In [ ]:
modgene_filt<-subset(modgene,module!="grey")
table(modgene_filt$module)

In [ ]:
for(i in unique(modgene_filt$module)){
    genes<-modgene_filt[modgene_filt$module==i,"gene"]
    da<-AddModuleScore(da,features = list(genes),name = i)
}
colnames(da@meta.data)

In [ ]:
module_score<-da@meta.data[,c(which(colnames(da@meta.data)=="coral1"):ncol(da@meta.data))]
colnames(module_score)<-gsub("1$","",colnames(module_score))
head(module_score)

In [93]:
write.csv(module_score,"WT.domain.thr100.sp18.hclust_average.min30.deep2.h015.33modules.SCT.score.ex_grey.csv")

In [ ]:
name<-colnames(da@meta.data)[which(colnames(da@meta.data)=="coral1"):ncol(da@meta.data)]
name
length(name)

In [95]:
#### adjust position to plot no-background, manual color ggplot
meta<-da@meta.data
#meta$x.ad2<-NA
#meta$y.ad2<-NA
#for(i in 1:length(unique(da2$orig.ident))){
#    m=(i-1)%/%4
#    x.add<-(i-4*m-1)*100
#    y.add<-150*(8-m)
#    meta[meta$orig.ident==unique(meta$orig.ident)[i],"x.ad2"]<-meta[meta$orig.ident==unique(meta$orig.ident)[i],"x.axis"]+x.add
#    meta[meta$orig.ident==unique(meta$orig.ident)[i],"y.ad2"]<-meta[meta$orig.ident==unique(meta$orig.ident)[i],"y.axis"]+y.add
#
#}
#range(meta$x.ad2)
#range(meta$y.ad2)

In [101]:
#dir.create("WTKO.SCT.hvg1w.union.cor.hcluster.L1norm.averagelinkage.filt30.module.scale.data.avg_spatialplot/")
sf<-"modulescore_spatial/"
if(!dir.exists(sf))
    dir.create(sf)
options(repr.plot.width=20,repr.plot.height=16)
for(i in name){
    ggplot(meta,aes_string("x.ad","y.ad",color=i))+geom_point(size=1)+
        #scale_colour_gradientn(colours=colorRampPalette(rev(brewer.pal(n = 11, name = "Spectral")))(50))+
        ggplot2::scale_colour_gradientn(colours = c('#0000ff','#3232ff','#6666ff','#9898ff','#ccccff',
                                              '#fffefe','#ffcccc','#ff9898','#ff6666','#ff3232','#ff0000'))+
        xlab(paste0("")) +
        ylab(paste0("")) + 
        theme(panel.background = element_blank(),
              panel.grid.major =element_blank(), 
              panel.grid.minor = element_blank(),
              axis.text.x = element_blank(),
              axis.text.y = element_blank()
          # axis.ticks.x = element_blank(),
    )#+
    #plot(p)
    #p<-SpatialFeaturePlot(mergda,features = c(i),min.cutoff=mean(mergda@meta.data[,i]),combine = F)
    #do.call(ggpubr::ggarrange,c(p,list(ncol=4,nrow=8,common.legend=T,legend="right")))
    ggsave(paste0(sf,"WT_",i,".module.score.spatial.png"),width=20,height=16,dpi=300)
    }

#### ridgeplot of each module at time_domain level

In [ ]:
df<-da@meta.data
colnames(df)<-gsub("1$","",colnames(df))
df$time<-factor(df$time,levels = rev(c("WT_sham","WT_3h","WT_24h","WT_72h")))
head(df)
levels(df$time)

In [140]:
library(ggridges)
library(ggsci)
library(scales)

In [ ]:
col<-c('#BC3C29A8','#0072B5A8','#E18727A8','#20854EA8')
show_col(col)
names(col)<-c('DH','MG','VH','WM')

In [ ]:
name<-gsub("1$","",name)
name

In [ ]:
options(repr.plot.width=6,repr.plot.height=4)
ggplot(df,aes_string(x="coral",y="time",color="domain",fill="domain"))+
    geom_density_ridges(color="white")+
    scale_y_discrete(expand = c(0,0))+
    scale_x_continuous(expand = c(0,0),name = paste0(i," module score")#,limits=c(min(df[,"lightcyan1"]),max(df[,"lightcyan1"]))
                      )+
    scale_fill_manual(values = c(alpha(col,0.66)),labels=c("DH","MG","VH","WM"))+
    coord_cartesian(clip = "off")+
    guides(fill=guide_legend(override.aes=list(fill=col,
                                               color=NA,points_color=NA)))+
    #ggtitle(paste0(i," module score at domain after SCI"))+
    theme_ridges(center=TRUE)+
    theme(panel.grid.major = element_blank(),axis.title.y.left = element_blank())

In [ ]:
options(repr.plot.width=6,repr.plot.height=4)
fig1<-"moduleScore_ridgePlot"
if(!dir.exists(fig1))
    dir.create(fig1)
for(i in name){
    ggplot(df,aes_string(x=i,y="time",color="domain",fill="domain"))+
    geom_density_ridges(color="white")+
    scale_y_discrete(expand = c(0,0))+
    scale_x_continuous(expand = c(0,0),name = paste0(i," module score")#,limits=c(min(df[,"lightcyan1"]),max(df[,"lightcyan1"]))
                      )+
    scale_fill_manual(values = c(alpha(col,0.66)),labels=c("DH","MG","VH","WM"))+
    coord_cartesian(clip = "off")+
    guides(fill=guide_legend(override.aes=list(fill=col,
                                               color=NA,points_color=NA)))+
    #ggtitle(paste0(i," module score at domain after SCI"))+
    theme_ridges(center=TRUE)+
    theme_ridges(center=TRUE)+
    theme(panel.grid.major = element_blank(),axis.title.y.left = element_blank())
    ggsave(paste0(fig1,"/",i,"_moduleScore.ridgePlot.png"),width = 6,height = 4,dpi = 300)
}
